In [287]:
import pandas as pd
import numpy as np
import datetime 
import math as mat
import time 
from datetime import timedelta
import matplotlib.pyplot as plt
import re 

In [288]:
accre_df=pd.read_csv('Data/july-2017-thru-june-2018.txt',sep="|",nrows=1000000)


In [290]:
accre_df.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,COMPLETED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21


In [4]:
accre_df['Elapsed'].head(6)

0    1-01:05:21
1      23:51:47
2    1-01:59:51
3    1-08:02:20
4      00:00:10
5      00:03:18
Name: Elapsed, dtype: object

### calculate the duration of a job
### job_duration=end_time-start_time
## Elapsed time is equal to difference between end and start time

In [291]:
start_time=accre_df['Start']
end_time=accre_df['End']
submit_time=accre_df['Submit']

In [292]:
print(start_time.head());print(end_time.head())

0    2017-07-03T02:13:59
1    2017-07-03T02:20:08
2    2017-07-03T02:41:13
3    2017-07-03T02:51:53
4    2017-07-01T05:00:01
Name: Start, dtype: object
0    2017-07-04T03:19:20
1    2017-07-04T02:11:55
2    2017-07-04T04:41:04
3    2017-07-04T10:54:13
4    2017-07-01T05:00:11
Name: End, dtype: object


# convert the start and end time from str to datetime format to calculate job duration

In [293]:
submit_time=pd.to_datetime(submit_time)

In [294]:
start_time=pd.to_datetime(start_time)

In [295]:
end_time=pd.to_datetime(end_time)

# calculate duration of a job

In [296]:
job_duration=(end_time-start_time)

In [297]:
job_duration=pd.DataFrame(job_duration)

In [298]:
job_duration.head()

,0
0,1 days 01:05:21
1,0 days 23:51:47
2,1 days 01:59:51
3,1 days 08:02:20
4,0 days 00:00:10


In [299]:
job_duration.shape

(1000000, 1)

In [300]:
accre_df.shape

(1000000, 25)

#  join the job_duration with accre_df

In [301]:

accre_df2=accre_df.join(job_duration)

In [302]:
accre_df2.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,0
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...,1 days 01:05:21
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...,0 days 23:51:47
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...,1 days 01:59:51
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...,1 days 08:02:20
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21,0 days 00:00:10



# rename the column of job_duration 

In [303]:

accre_df2.rename(columns={0:'job_duration'},inplace=True)

In [304]:
accre_df2.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,job_duration
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...,1 days 01:05:21
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...,0 days 23:51:47
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...,1 days 01:59:51
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...,1 days 08:02:20
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21,0 days 00:00:10


In [306]:
# make it dataframe
submit_time=pd.DataFrame(submit_time)

In [307]:
# replace the string value for sumbit time with datetime formated Submit_time
accre_df2['Submit']=submit_time

In [308]:
accre_df2.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName,job_duration
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02 00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...,1 days 01:05:21
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02 00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...,0 days 23:51:47
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02 00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...,1 days 01:59:51
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02 00:40:58,...,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...,1 days 08:02:20
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01 05:00:00,...,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21,0 days 00:00:10


In [309]:
accre_df2['Submit'].head()

0   2017-07-02 00:40:58
1   2017-07-02 00:40:58
2   2017-07-02 00:40:58
3   2017-07-02 00:40:58
4   2017-07-01 05:00:00
Name: Submit, dtype: datetime64[ns]

# check data type of job_duration column

In [310]:
accre_df2['job_duration'].head()




0   1 days 01:05:21
1   0 days 23:51:47
2   1 days 01:59:51
3   1 days 08:02:20
4   0 days 00:00:10
Name: job_duration, dtype: timedelta64[ns]

In [311]:
accre_select_df=accre_df2.loc[:,['Submit','User','JobID','job_duration']]
accre_select_df.head()


,Submit,User,JobID,job_duration
0,2017-07-02 00:40:58,masispider,15806649,1 days 01:05:21
1,2017-07-02 00:40:58,masispider,15806664,0 days 23:51:47
2,2017-07-02 00:40:58,masispider,15806670,1 days 01:59:51
3,2017-07-02 00:40:58,masispider,15806671,1 days 08:02:20
4,2017-07-01 05:00:00,uscms010,15811617,0 days 00:00:10


In [312]:
accre_select_df.Submit=pd.to_datetime(accre_select_df.Submit)

In [313]:
accre_select_df.job_duration=pd.to_timedelta(accre_select_df.job_duration)

In [314]:
accre_select_df.job_duration.dtype

dtype('<m8[ns]')

# sort the the accre_select_df by job_duration and user name in ascending order.

In [315]:
accre_select_df=accre_select_df.sort_values(by=['Submit','User','job_duration'],ascending=True)

In [316]:
accre_select_df.head()

,Submit,User,JobID,job_duration
4,2017-07-01 05:00:00,uscms010,15811617,00:00:10
5,2017-07-01 05:00:01,autocms,15811618,00:03:18
6,2017-07-01 05:00:02,tuos,15811619,00:02:03
7,2017-07-01 05:00:22,uscms010,15811621,00:00:10
8,2017-07-01 05:01:01,autocms,15811624,00:04:47


In [317]:
accre_select_df.job_duration.dtype

dtype('<m8[ns]')

In [318]:
accre_5_minutes= accre_select_df[accre_select_df.job_duration<'0 days 00:05:00']

In [320]:
accre_5_minutes.head(10)

,Submit,User,JobID,job_duration
4,2017-07-01 05:00:00,uscms010,15811617,00:00:10
5,2017-07-01 05:00:01,autocms,15811618,00:03:18
6,2017-07-01 05:00:02,tuos,15811619,00:02:03
7,2017-07-01 05:00:22,uscms010,15811621,00:00:10
8,2017-07-01 05:01:01,autocms,15811624,00:04:47
9,2017-07-01 05:01:02,tuos,15811625,00:03:12
31,2017-07-01 05:01:26,vuiiscci,15811649,00:00:00
33,2017-07-01 05:01:26,vuiiscci,15811651,00:00:00
34,2017-07-01 05:01:56,uscms010,15811654,00:02:00
35,2017-07-01 05:02:02,autocms,15811655,00:02:51


In [321]:
del(accre_5_minutes['job_duration'])

In [322]:
accre_5_minutes.head()

,Submit,User,JobID
4,2017-07-01 05:00:00,uscms010,15811617
5,2017-07-01 05:00:01,autocms,15811618
6,2017-07-01 05:00:02,tuos,15811619
7,2017-07-01 05:00:22,uscms010,15811621
8,2017-07-01 05:01:01,autocms,15811624


# group less than 5 minutes jobs into 4 hours range

In [325]:
accre_5_minutes['cum_count'] = accre_5_minutes.groupby(['User', pd.Grouper(freq='4H', key='Submit')]).cumcount()

C:\Users\fevty\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# users running more 500 jobs with in 4 hours range

In [326]:
accre_5_minutes_500=accre_5_minutes[accre_5_minutes.cum_count>500]

In [328]:
accre_5_minutes_500.head()

,Submit,User,JobID,cum_count
27715,2017-07-03 21:03:47,fletchs2,15845129,501
27716,2017-07-03 21:03:47,fletchs2,15845130,502
27717,2017-07-03 21:03:47,fletchs2,15845131,503
27718,2017-07-03 21:03:47,fletchs2,15845132,504
27719,2017-07-03 21:03:47,fletchs2,15845133,505


In [329]:
# check how many when Fletch2 exceeded 500 jobs
fletchs_2 =accre_5_minutes_500[accre_5_minutes_500.User =='fletchs2']

In [330]:
fletchs_2.head()

,Submit,User,JobID,cum_count
27715,2017-07-03 21:03:47,fletchs2,15845129,501
27716,2017-07-03 21:03:47,fletchs2,15845130,502
27717,2017-07-03 21:03:47,fletchs2,15845131,503
27718,2017-07-03 21:03:47,fletchs2,15845132,504
27719,2017-07-03 21:03:47,fletchs2,15845133,505


In [331]:
accre_5_minutes[['User','JobID']].groupby('User').agg('count')

,JobID
User,
albertme,21
anderkj2,9
annisj,20
arnoldjr,12
autocms,35786
barnekr1,25
bellcs1,5
beloinsb,1
bentonml,6


In [271]:
accre_5_minutes_500.groupby('User').max()

,Submit,JobID,job_duration,cum_count
User,,,,
bozhanng,2017-07-12 23:19:03,16130725_8,00:00:30,603
brockc,2017-07-21 15:30:31,16572675,00:04:59,841
cakira,2017-07-15 00:33:15,16305153,00:04:59,4592
cmsglidein,2017-07-28 13:17:48,16818357,00:04:59,5220
deik,2017-07-14 03:08:38,16223019_26,00:04:59,680
fletchs2,2017-07-15 02:06:51,16313302,00:04:23,4427
khancm,2017-07-13 19:54:33,16190547_997,00:04:59,2442
luchieac,2017-07-13 18:24:03,16186835_99,00:01:46,999
marlowb,2017-07-24 21:34:20,16672172,00:03:41,2377


result.head(50)